# Day 10: Syntax Scoring
The navigation subsystem syntax is made of several lines containing chunks. There are one or more chunks on each line, and chunks contain zero or more other chunks. Adjacent chunks are not separated by any delimiter; if one chunk stops, the next chunk (if any) can immediately start. Every chunk must open and close with one of four legal pairs of matching characters:

Find and discard the corrupted lines first.

A corrupted line is one where a chunk closes with the wrong character - that is, where the characters it opens and closes with do not form one of the four legal pairs listed above.

Examples of corrupted chunks include (], {()()()>, (((()))}, and <([]){()}[{}]). Such a chunk can appear anywhere within a line, and its presence causes the whole line to be considered corrupted.

In [ ]:
let sample = "[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]"

let inline (|Opener|Closer|Invalid|) string = //match char with '('|'['|'{'|'<' -> Opener char | ')'|']'|'}'|'>' -> Closer char | _ -> Invalid char
    match Seq.head string with
    | h when Array.contains h [|'(';'[';'{';'<'|] -> Opener (h, Seq.tail string)
    | h when Array.contains h [|')';']';'}';'>'|] -> Closer (h, Seq.tail string)
    | _ -> Invalid (' ', [' '])

let rec interpret (line:seq<char>) openBuffer =
    let h = Seq.head line
    match h with
    //  If it's any of the openers, add our head to the top of the queue and move deeper
    | '('|'['|'{'|'<' -> seq { yield h; yield! openBuffer } |> interpret (Seq.tail line)
    //  If it's any of our closers in which the head of our buffer is their cohort, unwrap a bit and move deeper
    | ')' when Seq.head openBuffer = '(' -> Seq.tail openBuffer |> interpret (Seq.tail line)
    | ']' when Seq.head openBuffer = '[' -> Seq.tail openBuffer |> interpret (Seq.tail line)
    | '}' when Seq.head openBuffer = '{' -> Seq.tail openBuffer |> interpret (Seq.tail line)
    | '>' when Seq.head openBuffer = '<' -> Seq.tail openBuffer |> interpret (Seq.tail line)
    //  Now we're in it
    | _ -> 1 //For now

"()" |> interpret
// sample.Split(Environment.NewLine) |> Array.item 2 |> interpret []


Stopped due to error


Error: input.fsx (30,1)-(30,18) typecheck error Value restriction. The value 'it' has been inferred to have generic type
    val it: (seq<char> -> int)    
Either make the arguments to 'it' explicit or, if you do not intend for it to be generic, add a type annotation.

In [ ]:
seq { yield 1; yield 2; yield 3;} |> Seq.head

1